In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
import pandas as pd
import torch
import numpy as np
import random
import re
import spacy
import shutil
import matplotlib.pyplot as plt
!pip install transformers
!pip install optuna
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, BertTokenizer, AutoModel, BertModel
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler
import time
import optuna
from scipy.stats import spearmanr
import psycopg2
import csv
import math
import os
from sqlalchemy import create_engine
from optuna.storages import RDBStorage
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
path_base = '/content/drive/MyDrive/Tesis/SICK/transformer_tunned_BERT/uncase_base/study_optuna/'
file_root = '/content/drive/MyDrive/Tesis/SICK/uncase_based/SICK.txt'
with open(file_root, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split("\t")) for l in temp_f.readlines() ]

In [ ]:
"""
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}
"""

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def getSICKSents(filename='sts-train.csv', set_type='TRAIN', encoding='utf-8'):
  f = open(filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for i, line in enumerate(f):
    example = re.split(r'\t+', line)
    if i >= 1:
      if example[-1].strip() == set_type:
        s2.append(example[1])
        s1.append(example[2])
        target.append(float(example[4]))
  print("samples: {}".format(len(target)))
  return s1, s2, target

In [5]:
s1_train,s2_train,target_train = getSICKSents(file_root, 'TRAIN')

samples: 4439


In [6]:
s1_test,s2_test,target_test= getSICKSents(file_root, 'TEST')

samples: 4906


In [8]:
s1_dev,s2_dev,target_dev= getSICKSents(file_root, 'TRIAL')

samples: 495


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cuda


In [10]:
BATCH_SIZE = 32
MAX_LEN = 128
CORPUS = 'SICK'
BERT_PATH = "bert-base-uncased"

In [11]:
TOKENIZER = BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def encode_sents(sents1, sents2):
    input_ids_ = []
    attention_masks_ = []
    type_ids_ = []
    for i, sent1 in enumerate(sents1):
        encoded_dict = TOKENIZER.encode_plus(
                            sent1,                      # Sentence 1 to encode.
                            sents2[i],                  # Sentence 2 to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            truncation = True,
                            max_length = MAX_LEN,       # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                      )

        # Add the encoded sentence to the list.
        input_ids_.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_.append(encoded_dict['attention_mask'])

        type_ids_.append(encoded_dict['token_type_ids'])
    return input_ids_, attention_masks_, type_ids_

In [13]:
def getEncodedTensors(s1, s2, labels):
    input_ids, attention_masks, type_ids = encode_sents(s1, s2)
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    type_ids = torch.cat(type_ids, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, type_ids, labels

In [14]:
input_ids_train,attention_masks_train,type_ids_train,labels_train = getEncodedTensors(s1_train,s2_train,target_train)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval = getEncodedTensors(s1_dev, s2_dev, target_dev)

In [16]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_eval, attention_masks_eval, type_ids_eval, labels_eval)

In [17]:
# Create the DataLoaders for our training and test sets.
# We'll take training samples in random order.
torch.manual_seed(42)
train_loader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = BATCH_SIZE # Trains with this batch size.
            #num_workers=2
        )
torch.manual_seed(42)
val_loader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = BATCH_SIZE # Evaluate with this batch size.
            #num_workers=2
        )

In [18]:
# See first batch
#batch = next(iter(train_loader))
#print(batch[0]) # 0 -> input_ids , 1 -> attention_masks, 2 -> type_ids, 3 -> targets
#print(train_loader.batch_size) #tamaño del batch
print('Number of train batches: {}'.format(len(train_loader)))# número de batches
print('Number of evaluation batches: {}'.format(len(val_loader)))

Number of train batches: 139
Number of evaluation batches: 16


In [19]:
class BertBaseUncasedRegressor(nn.Module):
  def __init__(self, num_classes = 1, dropout=0.1, attentions=True, n_layers = 0):
      super().__init__()
      self.bert = BertModel.from_pretrained(BERT_PATH, output_attentions=attentions) #load the model
      # Return:
      # last_hidden_state.shape -> [batch_size, num_tokens_in_sequence,hidden_size] (bert_base hidden_size = 768)
      # pooler_output.shape -> [batch_size, hidden_size] se utiliza una capa de pooling simple que aplica una transformación
      # lineal seguida de una función de activación tangente hiperbólica (tanh) a la última representación oculta (last_hidden_state) del token [CLS].
      layers = []
      for _ in range(n_layers):
          layers.append(nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)) #self.bert.config.hidden_size -> nos da el tamaño oculto
          layers.append(nn.Dropout(dropout))
      layers.append(nn.Linear(self.bert.config.hidden_size, num_classes)) #self.bert.config.hidden_size -> nos da el tamaño oculto

      self.regressor = nn.Sequential(*layers)


  def forward(self, input_ids, type_ids, mask):
      bert_output =self.bert(input_ids=input_ids, token_type_ids = type_ids, attention_mask= mask)
      #output = bert_output.pooler_output
      output = bert_output.last_hidden_state[:, 0, :]
      logits = self.regressor(output)
      #logits = self.regressor(output_drop)
      return logits, bert_output

In [20]:
def pearson_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)
    return np.corrcoef(all_preds, all_targets)[0, 1]

In [21]:
def spearman_corr(y_pred, y_true):
    all_preds = np.concatenate(y_pred)
    all_targets = np.concatenate(y_true)

    corr, _ = spearmanr(all_preds, all_targets)
    return corr

In [22]:
def train_loop(model,loader,optimizer,criterion,device,clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    all_preds = []
    all_targets = []
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

    for i, batch in enumerate(loader):

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        labels = batch[3].to(device)

        optimizer.zero_grad()
        #Forward
        outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
        outputs = outputs.squeeze(-1)
        #Loss
        loss = criterion(outputs.view(-1), labels.float())
        #Backprop
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()

        loss_sum += loss.item()

        all_preds.append(outputs.detach().cpu().numpy())
        all_targets.append(labels.float().detach().cpu().numpy())

    epoch_train_loss = loss_sum / len(loader)
    epoch_train_pearson = pearson_corr(all_preds, all_targets)
    epoch_train_spearman = spearman_corr(all_preds, all_targets)

    return epoch_train_loss, epoch_train_pearson, epoch_train_spearman

In [23]:
def evaluation_loop(model,loader,criterion,device):
    #Evaluation loop
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    model.eval()
    with torch.no_grad():
        loss_sum = 0
        all_preds = []
        all_targets = []

        for i, batch in enumerate(loader):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_type_ids = batch[2].to(device)
            labels = batch[3].to(device)

            #Forward
            outputs, _ = model(input_ids=b_input_ids, type_ids = b_type_ids, mask = b_attention_mask)
            outputs = outputs.squeeze(-1)
            #Loss
            loss = criterion(outputs.view(-1), labels.float())

            loss_sum += loss.item()

            all_preds.append(outputs.detach().cpu().numpy())
            all_targets.append(labels.float().detach().cpu().numpy())

        epoch_dev_loss = loss_sum / len(loader)
        epoch_dev_pearson = pearson_corr(all_preds, all_targets)
        epoch_dev_spearman = spearman_corr(all_preds, all_targets)

    return epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman

In [24]:
def save_metrics_dataframe(metrics_dict, study_name, path_base):
  # Abrir archivo en modo escritura y especificar el separador de campos
  path = path_base + 'metrics_' + study_name + ".csv"
  with open(path, "a+", newline="") as f:
      metrics = [metrics_dict]
      # Crear objeto escritor CSV y especificar el separador de campos
      metrics_csv = csv.DictWriter(f, fieldnames=['train_loss','valid_loss','spearman_train','spearman_val','pearson_train','pearson_val'],delimiter=",")

      # Verificar si se ha escrito el encabezado del archivo
      if f.tell() == 0:
        metrics_csv.writeheader()

      # Escribir cada fila de datos
      for metric in metrics:
          metrics_csv.writerow(metric)

In [25]:
study_name = 'study-bert-CLS-base_SICK'

In [26]:
def isNan(value):
  return 0 if math.isnan(value) else value

In [38]:
def objective(trial):
  params = {
      "num_layers": trial.suggest_int("num_layer", 0, 2),
      "dropout": trial.suggest_uniform("dropout", 0, 0.7),
      "optimizer": trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
      "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)
  }

  model = BertBaseUncasedRegressor(dropout=params["dropout"], n_layers = params["num_layers"]).to(device)
  criterion = nn.MSELoss()
  optimizer = getattr(optim, params["optimizer"])(model.parameters(), lr=params["learning_rate"])

  NUM_EPOCHS = 5
  best_valid_loss = float('inf')
  MODEL_FILE_NAME = CORPUS+'_'+BERT_PATH+'_'+str(MAX_LEN)+'_tunned_model.pt'
  history = {"train": {"loss": []}, "test": {"loss": []}}

  torch.cuda.empty_cache()
  seed = 42
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)

  train_loss_list, val_loss_list, spearman_train, spearman_val, pearson_train, pearson_val = [], [], [], [], [], []

  for epoch in range(NUM_EPOCHS):

    start_time = time.time()

    epoch_train_loss, epoch_train_pearson, epoch_train_spearman = train_loop(model,train_loader,optimizer,criterion,device)
    epoch_dev_loss, epoch_dev_pearson, epoch_dev_spearman = evaluation_loop(model,val_loader,criterion,device)

    elapsed_time = time.time() - start_time

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

    #nos quedamos con el modelo que tiene mejor pérdida de validación
    if epoch_dev_loss < best_valid_loss:
      best_valid_loss = epoch_dev_loss
      torch.save(model.state_dict(), MODEL_FILE_NAME)

    train_loss_list.append(isNan(epoch_train_loss))
    val_loss_list.append(isNan(epoch_dev_loss))
    spearman_train.append(isNan(epoch_train_spearman))
    spearman_val.append(isNan(epoch_dev_spearman))
    pearson_train.append(isNan(epoch_train_pearson))
    pearson_val.append(isNan(epoch_dev_pearson))

    history["train"]["loss"].append(epoch_train_loss)
    history["test"]["loss"].append(epoch_dev_loss)

    print('-' * 80)
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Time: {elapsed_time:.4f}s | Train loss: {epoch_train_loss:.4f} | Dev loss: {epoch_dev_loss:.4f}')
    print(f'Train Pearson Coef: {epoch_train_pearson:.4f} | Dev Pearson Coef: {epoch_dev_pearson:.4f}')
    print(f'Train Spearman Coef: {epoch_train_spearman:.4f} | Dev Spearman Coef: {epoch_dev_spearman:.4f}')

  metrics_dict = {
      'train_loss' : train_loss_list,
      'valid_loss' : val_loss_list,
      'spearman_train': spearman_train,
      'spearman_val': spearman_val,
      'pearson_train': pearson_train,
      'pearson_val': pearson_val
  }

  save_metrics_dataframe(metrics_dict, study_name, path_base)

  return best_valid_loss

In [34]:
storage_path = path_base + "optuna_study_CLS_SICK_.db"

In [35]:
# Define el estudio de Optuna utilizando el storage
study = optuna.create_study(study_name=study_name, direction="minimize", storage=f"sqlite:///{storage_path}", load_if_exists=True)

[I 2023-11-17 18:32:22,746] Using an existing study with name 'study-bert-CLS-base_SICK' instead of creating a new one.


In [39]:
study.optimize(objective, n_trials=15)

<ipython-input-38-72f2a3f0b480>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-38-72f2a3f0b480>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 87.2884s | Train loss: 0.7984 | Dev loss: 0.2651
Train Pearson Coef: 0.6103 | Dev Pearson Coef: 0.8653
Train Spearman Coef: 0.5600 | Dev Spearman Coef: 0.8270
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 87.9041s | Train loss: 0.2468 | Dev loss: 0.2541
Train Pearson Coef: 0.8788 | Dev Pearson Coef: 0.8754
Train Spearman Coef: 0.8228 | Dev Spearman Coef: 0.8311
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 88.0447s | Train loss: 0.1777 | Dev loss: 0.3616
Train Pearson Coef: 0.9146 | Dev Pearson Coef: 0.8742
Train Spearman Coef: 0.8828 | Dev Spearman Coef: 0.8271
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 88.2551s | Train loss: 0.1389 | Dev loss: 0.2600
Train Pearson Coef: 0.9336 | Dev Pearson Coef: 0.8819
Train Spearm

[I 2023-11-17 18:40:50,093] Trial 17 finished with value: 0.2541371029801667 and parameters: {'num_layer': 1, 'dropout': 0.6755169048801257, 'optimizer': 'Adam', 'learning_rate': 4.1450168177374585e-05}. Best is trial 5 with value: 0.21326825488358736.
<ipython-input-38-72f2a3f0b480>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "dropout": trial.suggest_uniform("dropout", 0, 0.7),
<ipython-input-38-72f2a3f0b480>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate",1e-5, 1e-4)


--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 87.0138s | Train loss: 0.6390 | Dev loss: 0.2755
Train Pearson Coef: 0.6816 | Dev Pearson Coef: 0.8773
Train Spearman Coef: 0.6158 | Dev Spearman Coef: 0.8322
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 87.0121s | Train loss: 0.1892 | Dev loss: 0.3895
Train Pearson Coef: 0.9063 | Dev Pearson Coef: 0.8888
Train Spearman Coef: 0.8619 | Dev Spearman Coef: 0.8443
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 86.9509s | Train loss: 0.1023 | Dev loss: 0.2900
Train Pearson Coef: 0.9505 | Dev Pearson Coef: 0.8890
Train Spearman Coef: 0.9248 | Dev Spearman Coef: 0.8484
--------------------------------------------------------------------------------
Epoch: 004/5 | Time: 87.0117s | Train loss: 0.0750 | Dev loss: 0.2188
Train Pearson Coef: 0.9641 | Dev Pearson Coef: 0.8911
Train Spearm

...

--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 88.2206s | Train loss: 0.0795 | Dev loss: 0.3005
Train Pearson Coef: 0.9622 | Dev Pearson Coef: 0.8855
Train Spearman Coef: 0.9465 | Dev Spearman Coef: 0.8427
--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 87.0459s | Train loss: 0.5670 | Dev loss: 0.3105
Train Pearson Coef: 0.7042 | Dev Pearson Coef: 0.8605
Train Spearman Coef: 0.6295 | Dev Spearman Coef: 0.8181
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 86.9586s | Train loss: 0.1991 | Dev loss: 0.4525
Train Pearson Coef: 0.9006 | Dev Pearson Coef: 0.8741
Train Spearman Coef: 0.8401 | Dev Spearman Coef: 0.8293
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 86.9221s | Train loss: 0.0988 | Dev loss: 0.4252
Train Pearson Coef: 0.9520 | Dev Pearson Coef: 0.8707
Train Spearm

[I 2023-11-17 20:17:13,315] Trial 30 finished with value: 0.3105046469718218 and parameters: {'num_layer': 0, 'dropout': 0.10661945091455358, 'optimizer': 'RMSprop', 'learning_rate': 1.0035136729014452e-05}. Best is trial 5 with value: 0.21326825488358736.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 86.9035s | Train loss: 0.0439 | Dev loss: 0.4327
Train Pearson Coef: 0.9791 | Dev Pearson Coef: 0.8690
Train Spearman Coef: 0.9694 | Dev Spearman Coef: 0.8154
--------------------------------------------------------------------------------
Epoch: 001/5 | Time: 84.4896s | Train loss: 12.0242 | Dev loss: 12.0253
Train Pearson Coef: 0.1855 | Dev Pearson Coef: 0.3649
Train Spearman Coef: 0.1598 | Dev Spearman Coef: 0.3067
--------------------------------------------------------------------------------
Epoch: 002/5 | Time: 84.5811s | Train loss: 11.8580 | Dev loss: 11.8489
Train Pearson Coef: 0.1901 | Dev Pearson Coef: 0.3689
Train Spearman Coef: 0.1633 | Dev Spearman Coef: 0.3095
--------------------------------------------------------------------------------
Epoch: 003/5 | Time: 84.4580s | Train loss: 11.6930 | Dev loss: 11.6739
Train Pearson Coef: 0.1946 | Dev Pearson Coef: 0.3727
Train 

[I 2023-11-17 20:24:27,846] Trial 31 finished with value: 11.327612817287445 and parameters: {'num_layer': 1, 'dropout': 0.24663942433001287, 'optimizer': 'SGD', 'learning_rate': 1.3958088554809765e-05}. Best is trial 5 with value: 0.21326825488358736.


--------------------------------------------------------------------------------
Epoch: 005/5 | Time: 84.4941s | Train loss: 11.3664 | Dev loss: 11.3276
Train Pearson Coef: 0.2034 | Dev Pearson Coef: 0.3798
Train Spearman Coef: 0.1733 | Dev Spearman Coef: 0.3163


In [40]:
df_optuna = study.trials_dataframe(attrs=("number", "value", "params", "state")).dropna(subset=['value']) #eliminar valores NaN
# Leer el archivo metrics.CSV y crear un DataFrame
path_metrics = path_base + 'metrics_' + study_name + ".csv"
df_metrics = pd.read_csv(path_metrics)
print(len(df_optuna))
print(len(df_metrics ))

30
30


In [42]:
df_old = pd.read_csv(path_base + study_name + "_COMPLETE_SORT.csv")
print("Old trials: ", len(df_old))

Old trials:  15


In [43]:
if len(df_optuna)-len(df_metrics) == len(df_old):
    df_optuna_filter = df_optuna.tail(len(df_metrics)).reset_index(drop=True)
else:
    # Seleccionar los valores de df_optuna que no están ya en df_old
    mask = ~df_optuna['value'].isin(df_old['value'])
    df_optuna_filter = df_optuna[mask].reset_index(drop=True)
    print(len(df_optuna_filter))

24


In [44]:
def concat(df_optuna_filter, df_metrics, df_old):
  # Unir dataframe de metrics con dataframe de optuna de forma horizontal y manejar los valores inexistentes
  df_join = pd.concat([df_optuna_filter, df_metrics], axis=1, sort=False)
  # Unir dataframe de metrics con dataframe de optuna de forma vertical y manejar los valores inexistentes
  df_complete_updated = pd.concat([df_old, df_join], axis=0, sort=False).drop_duplicates()
  print("All trials: ", len(df_complete_updated))
  assert len(df_join) + len(df_old) == len(df_complete_updated)
  return df_complete_updated

In [45]:
if len(df_optuna_filter) == len(df_metrics):
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss', 'value'])

if len(df_optuna_filter) > len(df_metrics):
  num_rows_to_add = len(df_optuna_filter) - len(df_metrics)
  new_rows = [[np.nan] * len(df_metrics.columns) for i in range(num_rows_to_add)]
  new_df = pd.DataFrame(new_rows, columns=df_metrics.columns)
  df_metrics = pd.concat([new_df, df_metrics], ignore_index=True)
  df_complete_updated = concat(df_optuna_filter, df_metrics, df_old).dropna(subset=['train_loss','value'])

In [46]:
if len(df_optuna) == len(df_metrics):
  df_complete_updated = pd.concat([df_optuna, df_metrics], axis=1, sort=False)

In [47]:
len(df_complete_updated)

32

In [48]:
# Ordenar el DataFrame unido por valor
df = df_complete_updated.sort_values(by='value', ascending=True)
# Obtener la prueba con el mejor valor
best_trial = df.iloc[0]
print("Best trial: ")
print(best_trial)

Best trial: 
number                                                                5.0
value                                                            0.213268
params_dropout                                                   0.110813
params_learning_rate                                             0.000065
params_num_layer                                                      3.0
params_optimizer                                                     Adam
state                                                            COMPLETE
train_loss              [0.7407780790286098, 0.2153756965300162, 0.185...
valid_loss              [0.2556082406081259, 0.3205687487497926, 0.230...
spearman_train          [0.6196575975707355, 0.8471167735836188, 0.873...
spearman_val            [0.8228808120206714, 0.8248442817009832, 0.835...
pearson_train           [0.6383390466053943, 0.8938386339815367, 0.911...
pearson_val             [0.8726601260544505, 0.8731576118196129, 0.883...
Name: 5, dtype: object


In [35]:
#Guardar CSV completo y ordenado
# ruta de acceso a la carpeta de Google Drive montada
#os.remove(path_base + study_name + "_COMPLETE_SORT.csv")
path_ = path_base + study_name + "_COMPLETE_SORT.csv"
# guardar el archivo CSV en Google Drive
df.to_csv(path_, index=False)
files.download(path_)
#os.remove(path_metrics)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>